In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import time
import pandas as pd

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Define a função para clicar no botão "Ver mais"
def click_ver_mais(driver):
    try:
        ver_mais_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver mais')]")
        if ver_mais_button.is_displayed():
            ver_mais_button.click()
            return True
    except:
        return False

# Abrir o navegador Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(r"C:\python312\chromedriver.exe"), options=options) 

# URL do Quinto Andar para casas à venda em São Paulo (exemplo)

driver.get("https://www.quintoandar.com.br/comprar/imovel/campinas-sp-brasil/apartamento/2-3-4-quartos/1-2-3-vagas/1-2-3-4-banheiros/de-150000-a-400000-venda")
time.sleep(5)

for filter in [4,5]:
    driver.find_element(By.ID, 'SORT_BUTTON').click()
    driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[2]/div/div/div/div/li[{filter}]').click()
    time.sleep(5)

    qnt_apt = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[1]/div/p/span').text
    qnt_apt = int(qnt_apt.replace('.',''))
    rows_apt = int(qnt_apt/3)+1
    col_last_line = qnt_apt%3

    df_apts = pd.read_csv('links_apartamentos.csv')

    for i in range (2,rows_apt+1):
        if len(df_apts)*100/qnt_apt >= 100: break
        for j in range(1,4):
            if i == rows_apt and j > col_last_line: break
            if (i-2)*3 + j >= qnt_apt*0.6: break
            try:
                link_apt = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[{i}]/div[{j}]/div/a').get_attribute("href")
                if link_apt not in df_apts['link'].tolist(): 
                    df_apts = pd.concat([df_apts, pd.DataFrame({'link': [link_apt]})], ignore_index=True)
                    df_apts.to_csv('links_apartamentos.csv', index=False)
                if len(df_apts)*100/qnt_apt >= 100: break
            except:
                print(f"{len(df_apts)*100/qnt_apt:.2f}% do total. {(((i-2)*3 + j)*100/qnt_apt):.2f}% desse filtro")
                scroll_to_bottom(driver)
                if click_ver_mais(driver):
                    # Espera um pouco para a nova lista de apartamentos carregar
                    time.sleep(5)
                else:
                    print("Não foi possível encontrar o botão 'Ver mais'.")
                    break  # Sai do loop interno
print(f"Extração concluida, foram encontrados links de {len(df_apts)} apartamentos")



driver.quit()


93.42% do total. 0.51% desse filtro
93.59% do total. 1.41% desse filtro
93.65% do total. 2.08% desse filtro
93.70% do total. 2.75% desse filtro
93.76% do total. 3.43% desse filtro
93.76% do total. 4.10% desse filtro
93.76% do total. 4.78% desse filtro
93.76% do total. 5.45% desse filtro
93.76% do total. 6.13% desse filtro
93.82% do total. 6.80% desse filtro
93.87% do total. 7.48% desse filtro
93.93% do total. 8.15% desse filtro
93.99% do total. 8.83% desse filtro
93.99% do total. 9.50% desse filtro
94.04% do total. 10.17% desse filtro
94.10% do total. 10.85% desse filtro
94.21% do total. 11.52% desse filtro
94.32% do total. 12.20% desse filtro
94.49% do total. 12.87% desse filtro
94.55% do total. 13.55% desse filtro
94.72% do total. 14.22% desse filtro
94.88% do total. 14.90% desse filtro
95.00% do total. 15.57% desse filtro
95.00% do total. 16.25% desse filtro
95.05% do total. 16.92% desse filtro
95.05% do total. 17.59% desse filtro
95.11% do total. 18.27% desse filtro
95.22% do total